# Tropical cyclone categories

This notebook provides a look-up table of the wind speed thresholds for different tropical cyclone categories. Given the different wind speed averaging periods, wind speed units and TC classification schemes, it can be confusing as to what the thresholds are for a category 2 cyclone in the SW Pacific compared to a tropical storm in the Atlantic, or a typhoon in the NW Pacific. 

Here, users can select the classification scheme (BoM, Saffir-Simpson, Typhoons), the input units (km/h, knots, m/s, miles/hr) and the wind speed averaging period (10-minute mean, 1-minute sustained, 3-second, 0.2 second gust) and have the categories plotted up.

The base classification uses a reference time corresponding to a 10-minute mean wind speed. We then use the World Meteorological Organisation's *Guidelines for Converting Between Various Wind Averaging Periods in Tropical Cyclone Conditions* ([WMO TD1555, 2010](http://www.wmo.int/pages/prog/www/tcp/documents/WMO_TD_1555_en.pdf)) to convert between different wind averaging periods. 


In [ ]:
%matplotlib inline
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, interactive, Dropdown
import ipywidgets as widgets
from collections import OrderedDict

We provide a range of time averaging periods - the values are in seconds, so for a 1-minute sustained wind, you would select `60.`

In [ ]:
tau = np.array([0.1, 0.2, .5, 1., 3., 10., 30., 60., 120., 180., 600., 3600.])

The wind speed averaging conversions are sensitive to the surface over which the wind is blowing -- by default, we will assume off-land conditions, but this can be changed. 

In [ ]:
turbulence = {'At-sea': 0.10,
              'Off-sea': 0.15,
              'Off-land': 0.20,
              'Inland': 0.25}

The base classifications are in knots, and we then convert to the chosen units:

In [ ]:
conversions = {"km/h":1.852,
               "mph":1.15,
               "m/s":0.5144,
               "kts":1.0}


These functions are used to determine the wind speed averaging conversion. 

In [ ]:
def crossingRate(tau, eta=10.):
    Tu = 3.13 * np.power(eta, 0.2)
    nu = (0.007 + 0.213 * np.power(Tu / tau, 0.654)) / Tu
    return nu

def gustFactor(tau, turbulence, reftime=3600., elev=10.):
    nu = crossingRate(tau, elev)
    Tu = 3.13 * np.power(elev, 0.2)
    sdratio = 1. - 0.193 * np.power((Tu / tau) + 0.1, -0.68)
    v_tau = np.sqrt(2. * np.log(reftime * nu)) + \
        0.577 / np.sqrt(2. * np.log(reftime * nu))
    g = v_tau * sdratio
    return 1. + g * turbulence

def esdu_gto(reftime):
    gto = 0.2193 * np.log(np.log10(reftime)) + 0.7242
    return gto

In [ ]:
palette = [(0.000, 0.627, 0.235), (0.412, 0.627, 0.235), 
           (0.663, 0.780, 0.282), (0.957, 0.812, 0.000), 
           (0.925, 0.643, 0.016), (0.835, 0.314, 0.118),
           (0.780, 0.086, 0.118)]

In [ ]:
categories = {
    'Australia and South Pacific':OrderedDict([
        ('Tropical low'                      ,{'min':  0., 'max': 33.}),
        ('Category 1 tropical cyclone'       ,{'min': 34., 'max': 47.}),
        ('Category 2 tropical cyclone'       ,{'min': 48., 'max': 63.}),
        ('Category 3 severe tropical cyclone',{'min': 64., 'max': 85.}),
        ('Category 4 severe tropical cyclone',{'min': 86., 'max':107.}),
        ('Category 5 severe tropical cyclone',{'min':108., 'max':300.})]
    ),
    'Saffir-Simpson':OrderedDict([
        ('Tropical Depression'       ,{'min':  0., 'max': 29.}),
        ('Tropical Storm'            ,{'min': 30., 'max': 55.}),
        ('Category 1 hurricane'      ,{'min': 56., 'max': 72.}),
        ('Category 2 hurricane'      ,{'min': 73., 'max': 83.}),
        ('Category 3 major hurricane',{'min': 84., 'max': 98.}),
        ('Category 4 major hurricane',{'min': 99., 'max':119.}),
        ('Category 5 major hurricane',{'min':120., 'max':300.})]
    ),
    'JTWC NW Pacific'  :OrderedDict([
        ('Tropical Depression',{'min':  0., 'max': 29.}),
        ('Tropical Storm'     ,{'min': 30., 'max': 55.}),
        ('Typhoon'            ,{'min': 56., 'max':113.}),
        ('Super Typhoon'      ,{'min':114., 'max':300.})]
    ),
    'Japanese Meteorological Agency':OrderedDict([
        ('Tropical Depression'  ,{'min':  0., 'max': 33.}),
        ('Tropical Storm'       ,{'min': 34., 'max': 47.}),
        ('Severe Tropical Storm',{'min': 48., 'max': 63.}),
        ('Typhoon'              ,{'min': 64., 'max':300.})]
    ),
    'Meteo France':OrderedDict([
        ('Tropical Disturbance'         ,{'min':  0., 'max': 29.}),
        ('Tropical Depression'          ,{'min': 30., 'max': 33.}),
        ('Moderate Tropical Storm'      ,{'min': 34., 'max': 47.}),
        ('Severe Tropical Storm'        ,{'min': 48., 'max': 63.}),
        ('Tropical Cyclone'             ,{'min': 64., 'max': 85.}),
        ('Intense Tropical Cyclone'     ,{'min': 86., 'max':113.}),
        ('Very Intense Tropical Cyclone',{'min':114., 'max':300.})]
    )
    
}

In [ ]:
def plot_classification(classification, avgper, sfccond, units):
    cat = categories[classification]
    cf = conversions[units]
    eta = 10.
    gto = esdu_gto(600.)
    
    gf = gustFactor(avgper, turbulence[sfccond], 600.) * gto
    columns = ['Category', 'Minimum', 'Maximum']
    nrows = len(cat.keys())
    ncols = len(columns)
    hcell, wcell = 1., 1.
    hpad, wpad = .2, 0.5
    cmap = sns.blend_palette(palette,nrows+1)

    fig, ax = plt.subplots(figsize=(3*(ncols*wcell+wpad), (nrows*hcell+hpad)))
    ax.axis('off')
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    for sp in ax.spines.itervalues():
        sp.set_color('w')
        sp.set_zorder(0)
    
    cell_text = [columns]
    cell_colors = [['w', 'w', 'w']]
    n = 1
    for label, val in cat.iteritems():
        minval = val['min'] * gf * cf
        maxval = val['max'] * gf * cf
        if val['min'] == 0.0:
            cell_text.append([label, '', np.round(maxval)])
        elif val['max'] == 300.:
            cell_text.append([label, np.round(minval), ''])
        else:
            cell_text.append([label, np.round(minval), np.round(maxval)])
        cell_colors.append([cmap[n], cmap[n], cmap[n]])
        n+=1
        
    the_table = plt.table(cellText=cell_text, loc='center', fontsize=24, cellColours=cell_colors)
    the_table.set_zorder(10)
    the_table.auto_set_font_size(False)
    the_table.set_fontsize(24)
    the_table.scale(2, 4)
    ax.set_title("{0} tropical cyclone classification ({1})".format(classification, units), fontsize=24)
    plt.show()
    

In [ ]:
catDropDown = Dropdown(options=categories.keys(), value=categories.keys()[0], description='Classification')
tauDropDown = Dropdown(options=list(tau), value=tau[10], description='Averaging period (s)')
turDropDown = Dropdown(options=turbulence.keys(), value=turbulence.keys()[2], description='Surface conditions')
unitDropDown = Dropdown(options=conversions.keys(), value=conversions.keys()[-1], description='Units')

In [ ]:
interact(plot_classification,
                classification=catDropDown,
                avgper = tauDropDown,
                sfccond = turDropDown,
                units = unitDropDown)
